<a href="https://colab.research.google.com/github/Shifa1729/-voice_cloning.ipynb/blob/README.md/AI_ASSESMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PIPELINE:
1. Enter youtube url.
2. whisper transcript the audio- single JSON file you can edit.
3. Audio segments are cancatenated and then split into sentence using nltk.
4. sentence are combine into < 30 second clip, one row per clip (audio,test source)
5. The resulting dataset.Dataset is pushed to Hugging face under the org/repo of your choice.

In [ ]:
!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=7043ebde88c0562525a9a85cd7389b141bc761c4b075f2807d9626879894a01e
  Stored in directory: /root/.cache/pip/wheels/34/b8/4e/9c4c3351d670e06746a340fb4b7d854c76517eec225e5b32b1
Successfully built whisper


In [6]:
# Installation and Imports
# Uninstall potentially conflicting whisper packages
!pip uninstall -y whisper whisper-cpp-python

# Install required packages including OpenAI whisper
!pip install -q ffmpeg-python sound dataset youtube-dl whisper nltk yt-dlp

# IMPORTS
import os, json, subprocess, uuid, torch, whisper, glob
from pathlib import Path
import nltk
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.12.0 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but yo

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# WHISPER MODEL AND HARDWARE
# Ensure the correct whisper is imported and loaded
import whisper
model = whisper.load_model("base")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

DOWNLOAD AUDIO FROM YOUTUBE

In [ ]:
  from google.colab import files
  uploaded = files.upload()
  #dialog lets you pick multiple files
  for name, _ in uploaded.items():
    dest = AUDIO_DIK/name
    os.rename(name, dest)
    print("save",dest)
  print ("all uploads saved to",AUDIO_DIR)

In [22]:
from os.path import exists
from sys import stdout
from urllib.error import URLError
import subprocess # Added import for subprocess
from pathlib import Path # Assuming Path is used and needs importing

#ADD / paste any number of Youtube links here
YOUTUBE_URLS = ["https://www.youtube.com/watch?v=dQw4w9WgXcQ",
                 "https://www.youtube.com/watch?v=mWZ6b_I-Djg",
                 "https://www.youtube.com/watch?v=B0p5SdkBydU",
                 "https://www.youtube.com/watch?v=xJbeStHI2HU"
                 ]
# Assuming AUDIO_DIR is defined elsewhere
# for url in Youtube_URLS # Original line with syntax error

for url in YOUTUBE_URLS: # Corrected for loop
    expected_filename = None
    # Attempt to get the title using yt.dlp's --print option
    try:
        process = subprocess.run(
            ["yt-dlp", "--print", "%(title)s", url],
            capture_output=True, # Corrected argument name
            text=True,
            check=True,
        )
        title = process.stdout.strip() # Corrected attribute access
        # Assuming AUDIO_DIR is a Path object
        # Construct expected filename
        expected_filename = Path(str(AUDIO_DIR) + "/" + title + ".m4a")

    except subprocess.CalledProcessError as e: # Corrected exception handling syntax
        print(f" warning: could not get title for {url}: {e}")
        # fallback or skip if filename cannot be determined
        print(f" downloading {url} (without checking for existing file)")
        # Continue with download even if title could not be retrieved
        subprocess.run(
            [
                "yt-dlp",
                "-q", "-x", "--extract-audio", "--audio-format", "m4a",
                "-o", str(AUDIO_DIR / "%(title)s.%(ext)s"),
                url,
            ],
            check=True
        )
        continue # Move to next url

    # If title was successfully retrieved, check if file exists before downloading
    if expected_filename and expected_filename.exists():
        print(f"File for {url} already exists: {expected_filename}")
        continue # Move to next url

    print(f"Downloading {url}")
    # Download the audio
    subprocess.run(
        [
            "yt-dlp",
            "-q", "-x", "--extract-audio", "--audio-format", "m4a",
            "-o", str(AUDIO_DIR / "%(title)s.%(ext)s"),
            url,
        ],
        check=True
    )
print("All Youtube audio save to", AUDIO_DIR)

All Youtube audio save to audio


UPLOAD AUDIO FILE

In [ ]:
import subprocess, json
from itertools import chain # Corrected import name
from pathlib import Path

def get_duration_sec(path:Path)->float:
  """Return length in seconds using ffprobe."""
  pr = subprocess.run(
      ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "json", str(path)],
      capture_output=True,
      check=True,
  )
  try:
      return float(json.loads(pr.stdout)["format"]["duration"]) # Corrected json.loads call
  except (json.JSONDecodeError, KeyError, ValueError) as e: # Corrected exception handling syntax and names
      print(f"error getting duration of {path}:{e}")
      print(f"ffprobe stdout: {pr.stdout}")
      print(f"ffprobe stderr: {pr.stderr}")
      return 0.0

#collect audio file
audio_paths = sorted(chain(
    AUDIO_DIR.glob("*.m4a"),
    AUDIO_DIR.glob("*.mp3"),
    AUDIO_DIR.glob("*.wav"),
    AUDIO_DIR.glob("*.webm"), # Corrected file extension
))

print (f"Found {len(audio_paths)} audio files") # Corrected f-string syntax
total_sec = 0
for p in audio_paths: # Corrected variable name
    dur_sec = get_duration_sec(p) # Corrected function name
    total_sec += dur_sec
    print(f"{p.name:<40} {dur_sec/60:6.2f} min")
print(f"total duration: {total_sec/60:6.2f} min"
    f"({total_sec/3600:.2f}h)") # Corrected hours calculation